# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path= "../output_data/cities.csv"

data= pd.read_csv(csv_path)

df= pd.DataFrame(data)

df.head()



,Unnamed: 0,Cities,Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Country,Date
0,0,Toora-Khem,21.16,88,3,2.28,52.4675,96.1100,RU,1634513483
1,1,Nanortalik,35.28,89,99,1.36,60.1432,-45.2371,GL,1634513486
2,2,Hilo,78.87,70,75,5.75,19.7297,-155.0900,US,1634513238
3,3,Avera,58.30,50,0,5.59,33.1940,-82.5271,US,1634513490
4,4,Geraldton,55.81,76,0,12.66,-28.7667,114.6000,AU,1634513493


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations= df[["Lat","Lng"]]

layout={'width': '1050px', 'height': '700px', 'padding':'3px', 'border': '1px solid black'}

fig= gmaps.figure(zoom_level=2, center=(25,25), layout=layout)

humidity= df['Humidity'].astype(float)

heat_layer=gmaps.heatmap_layer(locations, weights= humidity, dissipating=True, point_radius=12, max_intensity=100)



fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1050px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:



#filter down to perfect temps
ideal_temp= df.loc[(df['Temp'] < 85) & (df['Temp'] > 65), :]

#filter down to perfect windspeed
ideal_windspeed=  ideal_temp.loc[ideal_temp['Windspeed'] < 20, :]

#filter down to perfect cloudiness
ideal_cloudiness= ideal_windspeed.loc[ideal_windspeed['Cloudiness'] == 0,:]

perfect_weather= ideal_cloudiness





### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df= perfect_weather

hotel_df

,Unnamed: 0,Cities,Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Country,Date
12,12,Parsons,72.84,50,0,10.07,37.3403,-95.2611,US,1634513516
28,28,Ruwi,76.69,49,0,1.14,23.5996,58.5507,OM,1634513557
61,61,Valparaíso,65.84,45,0,4.61,-33.0393,-71.6273,CL,1634513455
206,206,Jalu,68.40,44,0,7.38,29.0331,21.5482,LY,1634513994
287,287,Marsá Maţrūḩ,66.40,77,0,3.44,31.3525,27.2453,EG,1634514202
366,366,Saint-Pierre,67.91,71,0,4.61,-21.3393,55.4781,RE,1634514399
396,396,Angoche,75.36,80,0,11.18,-16.2325,39.9086,MZ,1634514474
410,410,Aswān,81.70,19,0,12.66,24.0934,32.9070,EG,1634514489
494,494,Eilat,73.24,64,0,18.41,29.5581,34.9482,IL,1634514719
522,522,Quelimane,77.92,69,0,10.00,-17.8786,36.8883,MZ,1634514789


In [6]:
#find hotels with places api

hotel_names= []

hotel_lat= []

hotel_lng= []

radius= 50000
Type= "lodging"
key= g_key
for index, row in hotel_df.iterrows():
    
    lat= row['Lat']
    lng= row['Lng']
    location=f"{lat},{lng}"
  
    params= {"location":location, "radius":radius, "type": Type, "key":key}

    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response= requests.get(base_url, params=params)

    response= response.json()
    
    hotel_names.append(response['results'][0]["name"])
    
    hotel_lat.append(response['results'][0]['geometry']['location']['lat'])
    
    hotel_lng.append(response['results'][0]['geometry']['location']['lng'])
    

In [7]:
#add hotel names, lat's and lng's to hotel_df

hotel_df['Hotel Name']= hotel_names
hotel_df['Hotel Lat']= hotel_lat
hotel_df['Hotel Lng']= hotel_lng

hotel_df= hotel_df.rename(columns={"Cities":"City"})
hotel_df

<ipython-input-7-c27f6a907ee4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= hotel_names
<ipython-input-7-c27f6a907ee4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Lat']= hotel_lat
<ipython-input-7-c27f6a907ee4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

,Unnamed: 0,City,Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
12,12,Parsons,72.84,50,0,10.07,37.3403,-95.2611,US,1634513516,Parsons Inn,37.340958,-95.235885
28,28,Ruwi,76.69,49,0,1.14,23.5996,58.5507,OM,1634513557,Park Inn by Radisson Muscat,23.594386,58.413386
61,61,Valparaíso,65.84,45,0,4.61,-33.0393,-71.6273,CL,1634513455,Hotel Ultramar,-33.048796,-71.632399
206,206,Jalu,68.40,44,0,7.38,29.0331,21.5482,LY,1634513994,صاحبة الفخامة,29.032355,21.549665
287,287,Marsá Maţrūḩ,66.40,77,0,3.44,31.3525,27.2453,EG,1634514202,Carols Beau Rivage Hotel - فندق كارولز بوريفاج,31.393093,27.064449
366,366,Saint-Pierre,67.91,71,0,4.61,-21.3393,55.4781,RE,1634514399,Hôtel Le chenets,-21.135848,55.479259
396,396,Angoche,75.36,80,0,11.18,-16.2325,39.9086,MZ,1634514474,Guest House HESADA,-16.225924,39.905828
410,410,Aswān,81.70,19,0,12.66,24.0934,32.9070,EG,1634514489,Nuba Nile,24.100818,32.901494
494,494,Eilat,73.24,64,0,18.41,29.5581,34.9482,IL,1634514719,Club Hotel Eilat,29.549450,34.952215
522,522,Quelimane,77.92,69,0,10.00,-17.8786,36.8883,MZ,1634514789,Hotel Flamingo,-17.880454,36.887034


In [8]:
#Create info box content using iterrows

hotel_info_itr= []

for index, row in hotel_df.iterrows():
    
    name= row['Hotel Name']
    city= row['City']
    country= row['Country']
    
    hotel_info_itr.append(f'HOTEL NAME: {name} | CITY: {city} | COUNTRY: {country}')

In [9]:
# Add marker layer ontop of heat map

locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

markers= gmaps.marker_layer(locations, info_box_content= hotel_info_itr)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1050px'))